In [1]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont

ROOT = Path.cwd()
if not (ROOT / "pkg").exists():
    if (ROOT.parent / "pkg").exists():
        ROOT = ROOT.parent
    else:
        raise RuntimeError("Не найден корень репозитория (папка pkg)")

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from pkg.teeth_detector import detect_teeth
from pkg.metadata import compute_metadata
from pkg.teeth_classifier import load_classifier, predict_teeth

In [2]:
IMAGE_FOLDER = ROOT / "experiments" / "data" / "images"
OUTPUT_FOLDER = ROOT / "experiments" / "results" / "teeth_classifier"

BBOX_COLOR = (0, 255, 0)
BBOX_THICKNESS = 2
TEXT_COLOR = (0, 255, 0)
FONT_SIZE = 14
TEXT_OFFSET = 3

IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

In [3]:
print("Загрузка модели классификации...")
model, scaler, ohe, label_encoder = load_classifier()
print("✓ Модель загружена")

Загрузка модели классификации...
✓ Модель загружена


In [4]:
def get_image_files(folder_path) -> list:
    """Получение списка всех изображений в папке."""
    folder = Path(folder_path)
    image_files = []
    
    for ext in IMAGE_EXTENSIONS:
        image_files.extend(folder.glob(f'*{ext}'))
        image_files.extend(folder.glob(f'*{ext.upper()}'))
    
    return sorted(image_files)


def draw_bbox_with_label(image: np.ndarray, bbox: list, label: int, 
                          color: tuple = BBOX_COLOR, 
                          thickness: int = BBOX_THICKNESS) -> np.ndarray:
    """Отрисовка bounding box с подписью номера зуба."""
    img_pil = Image.fromarray(image)
    draw = ImageDraw.Draw(img_pil)
    
    x1, y1, x2, y2 = bbox[:4]
    
    draw.rectangle([x1, y1, x2, y2], outline=color, width=thickness)
    
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", FONT_SIZE)
    except:
        font = ImageFont.load_default()
    
    label_text = str(label)
    bbox_text = draw.textbbox((0, 0), label_text, font=font)
    text_width = bbox_text[2] - bbox_text[0]
    text_height = bbox_text[3] - bbox_text[1]
    
    text_x = x1
    text_y = max(0, y1 - text_height - TEXT_OFFSET)
    
    draw.rectangle(
        [text_x, text_y, text_x + text_width + 2, text_y + text_height + 2],
        fill=(0, 0, 0, 128)
    )
    
    draw.text((text_x + 1, text_y + 1), label_text, fill=color, font=font)
    
    return np.array(img_pil)


def process_image(image_path: str, model, scaler, ohe, label_encoder) -> dict:
    """Полный пайплайн обработки одного изображения."""
    image = cv2.imread(str(image_path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_width = image_rgb.shape[1]
    
    bboxes_full = detect_teeth(image_rgb)
    bboxes = [bbox[:4] for bbox in bboxes_full]
    
    if not bboxes:
        print(f"⚠ Зубы не обнаружены на {image_path}")
        return {'bboxes': [], 'labels': [], 'image': image_rgb, 'success': False}
    
    metadata_list = compute_metadata(bboxes, image_width)
    
    labels = predict_teeth(metadata_list, model, scaler, ohe, label_encoder)
    
    result_image = image_rgb.copy()
    for bbox, label in zip(bboxes, labels):
        result_image = draw_bbox_with_label(result_image, bbox, label)
    
    return {
        'bboxes': bboxes,
        'labels': labels,
        'image': result_image,
        'success': True
    }

In [5]:
image_files = get_image_files(IMAGE_FOLDER)
print(f"Найдено изображений: {len(image_files)}")

if len(image_files) == 0:
    print("⚠ Изображения не найдены! Проверьте путь IMAGE_FOLDER")

Найдено изображений: 4


In [6]:
Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

results = []

for i, img_path in enumerate(image_files):
    print(f"\n[{i+1}/{len(image_files)}] Обработка: {img_path.name}")
    
    result = process_image(img_path, model, scaler, ohe, label_encoder)
    results.append({
        'path': img_path,
        'result': result
    })
    
    if result['success']:
        output_path = Path(OUTPUT_FOLDER) / f"result_{img_path.stem}{img_path.suffix}"
        cv2.imwrite(str(output_path), cv2.cvtColor(result['image'], cv2.COLOR_RGB2BGR))
        print(f"  ✓ Найдено зубов: {len(result['labels'])}")
        print(f"  ✓ Сохранено: {output_path}")
        print(f"  ✓ Номера зубов: {result['labels']}")
    else:
        print(f"  ⚠ Обработка не удалась")

print(f"\n{'='*50}")
print(f"Обработка завершена! Результаты в папке: {OUTPUT_FOLDER}")


[1/4] Обработка: ct1.jpg


C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


  ✓ Найдено зубов: 19
  ✓ Сохранено: C:\Users\volgi\teeth_disease_detector\experiments\results\teeth_classifier\result_ct1.jpg
  ✓ Номера зубов: [47, 27, 44, 21, 36, 23, 35, 34, 12, 24, 13, 22, 45, 42, 33, 32, 42, 31, 28]

[2/4] Обработка: ct1.jpg


C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


  ✓ Найдено зубов: 19
  ✓ Сохранено: C:\Users\volgi\teeth_disease_detector\experiments\results\teeth_classifier\result_ct1.jpg
  ✓ Номера зубов: [47, 27, 44, 21, 36, 23, 35, 34, 12, 24, 13, 22, 45, 42, 33, 32, 42, 31, 28]

[3/4] Обработка: ct2.jpg


C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


  ✓ Найдено зубов: 32
  ✓ Сохранено: C:\Users\volgi\teeth_disease_detector\experiments\results\teeth_classifier\result_ct2.jpg
  ✓ Номера зубов: [37, 47, 48, 46, 16, 38, 14, 25, 26, 35, 27, 28, 23, 18, 15, 13, 36, 22, 33, 12, 11, 45, 24, 34, 21, 17, 44, 42, 43, 41, 31, 32]

[4/4] Обработка: ct2.jpg


C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\volgi\anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


  ✓ Найдено зубов: 32
  ✓ Сохранено: C:\Users\volgi\teeth_disease_detector\experiments\results\teeth_classifier\result_ct2.jpg
  ✓ Номера зубов: [37, 47, 48, 46, 16, 38, 14, 25, 26, 35, 27, 28, 23, 18, 15, 13, 36, 22, 33, 12, 11, 45, 24, 34, 21, 17, 44, 42, 43, 41, 31, 32]

Обработка завершена! Результаты в папке: C:\Users\volgi\teeth_disease_detector\experiments\results\teeth_classifier


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
axes = axes.flatten()

for i, result_data in enumerate(results[:4]):
    if result_data['result']['success']:
        axes[i].imshow(result_data['result']['image'])
        axes[i].set_title(f"{result_data['path'].name}\nЗубов: {len(result_data['result']['labels'])}")
    else:
        axes[i].text(0.5, 0.5, "Нет результатов", ha='center', va='center')
        axes[i].set_title(result_data['path'].name)
    axes[i].axis('off')

plt.tight_layout()
plt.show()